<a href="https://colab.research.google.com/github/tvaditya/AndroidProjects/blob/master/Sentimental_Analysis_With_Bert_as_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etapa 1: Importação das bibliotecas

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
from google.colab import drive

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 5.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=e049e07d19521ca33036d1e37a0c053b3e9b559c80db82fc65b62986e27dcebb
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=4ff7fdc6f7726896ad308f10def059192c1860f195c05f70b3ca50cb7c4a3cf3
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.1-cp36-none-any.whl size=16121 sha256=803d0b3558dfc49db4410eada8976d72e6b9ed42a5afadd80969c93ec0718c7b
  Stored in directory: /root/.cache/pip/wheels/d2/59/d0/ba322508e9593c78115ef46cf9dcf7f7d8a3fa9fc9f55c7bf3
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.0MB 17.0MB/s 


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
tf.__version__
# !pip install tensorflow==2.2.0-rc3

'2.2.0-rc3'

# Etapa 2: Pré-processamento

## Carregamento dos arquivos

In [ ]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv('/content/drive/My Drive/Base de dados sentimentos.zip (Unzipped Files)/Base de dados sentimentos/training.1600000.processed.noemoticon.csv',
                   header = None,
                   names = cols,
                   engine='python',
                   encoding='latin1')

In [ ]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [ ]:
data.shape

(1600000, 2)

In [ ]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## Pré-processamento

### Limpeza

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenização

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def encode_sentence(sent):
  return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [ ]:
encode_sentence("My dog likes strawberries.")

['[CLS]', 'my', 'dog', 'likes', 'straw', '##berries', '.', '[SEP]']

In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [ ]:
print(data_inputs[0:2])

[['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]'], ['[CLS]', 'is', 'upset', 'that', 'he', 'can', "'", 't', 'update', 'his', 'facebook', 'by', 'text', '##ing', 'it', '.', '.', '.', 'and', 'might', 'cry', 'as', 'a', 'result', 'school', 'today', 'also', '.', 'blah', '!', '[SEP]']]


### Criação da base de dados

In [ ]:
def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
get_ids(tokenizer.tokenize("My dog likes strawberries."))

[2026, 3899, 7777, 13137, 20968, 1012]

In [ ]:
np.char.not_equal("[PAD]", "[PAD]")

array(False)

In [ ]:
def get_mask(tokens):
  return np.char.not_equal(tokens, "[PAD]").astype(int)

In [ ]:
get_mask(tokenizer.tokenize("My dog likes strawberries."))

array([1, 1, 1, 1, 1, 1])

In [ ]:
def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0
  for tok in tokens:
    seg_ids.append(current_seg_id)
    if tok == "[SEP]":
      current_seg_id = 1 - current_seg_id
  return seg_ids

In [ ]:
print(data_inputs[0])

['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]']


In [ ]:
get_segments(data_inputs[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are red.") + ["[SEP]"]
my_sent

['[CLS]', 'roses', 'are', 'red', '.', '[SEP]']

In [ ]:
bert_layer([
            tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0) 
           ])

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.27935660e-01, -4.10335451e-01, -9.65755105e-01,
          9.07317996e-01,  8.12913895e-01, -1.74174488e-01,
          9.11234617e-01,  3.41952324e-01, -8.74521315e-01,
         -9.99989271e-01, -7.78410017e-01,  9.69385266e-01,
          9.86160517e-01,  6.36963248e-01,  9.48631346e-01,
         -7.51193345e-01, -4.58339751e-01, -7.08104610e-01,
          4.62098449e-01, -6.57927275e-01,  7.60414600e-01,
          9.99994814e-01, -3.96861047e-01,  3.44166338e-01,
          6.16488695e-01,  9.94400024e-01, -7.76633859e-01,
          9.38316584e-01,  9.59452331e-01,  7.32879400e-01,
         -6.93436861e-01,  2.93080568e-01, -9.93785501e-01,
         -1.64551944e-01, -9.67019558e-01, -9.95549560e-01,
          5.32935560e-01, -6.88061118e-01,  1.34714106e-02,
          2.98194252e-02, -9.18356597e-01,  4.20526385e-01,
          9.99988914e-01,  2.52676487e-01,  6.06235623e-01,
         -3.50750178e-01, -1.00000000e+00,  4.975

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key = lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
               get_mask(sent_lab[0]),
               get_segments(sent_lab[0])],
              sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [ ]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [ ]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Etapa 3: Construção do modelo

In [ ]:
class DCNNBERTEmbedding(tf.keras.Model):
    
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                                         trainable = False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def embed_with_bert(self, all_tokens):
      _, embs = self.bert_layer([all_tokens[:, 0, :],
                                 all_tokens[:, 1, :],
                                 all_tokens[:, 2, :]])
      return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)
        
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Etapa 4: Treinamento

In [ ]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2
BATCH_SIZE = 32
NB_EPOCHS = 5

In [ ]:
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [ ]:
checkpoint_path = "/content/drive/My Drive/Cursos - recursos"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [ ]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [ ]:
history = Dcnn.fit(train_dataset,
                   epochs=NB_EPOCHS,
                   callbacks=[MyCustomCallback()])

Epoch 1/5
  40623/Unknown - 2999s 74ms/step - loss: 0.3973 - accuracy: 0.8219Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 3001s 74ms/step - loss: 0.3973 - accuracy: 0.8219
Epoch 2/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3756 - accuracy: 0.8339Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2987s 74ms/step - loss: 0.3756 - accuracy: 0.8339
Epoch 3/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3659 - accuracy: 0.8391Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2979s 73ms/step - loss: 0.3659 - accuracy: 0.8391
Epoch 4/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3586 - accuracy: 0.8426Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2969s 73ms/step - loss: 0.3586 - accuracy: 0.8426
Epoch 5/5
40623/40623 [==============================] - ETA: 0s - loss: 0.3527 - accuracy: 0.8454Checkpoint saved at /content/drive/My Drive/Cursos - recursos.
40623/40623 [==============================] - 2960s 73ms/step - loss: 0.3527 - accuracy: 0.8454
<tensorflow.python.keras.callbacks.History at 0x7fd363ef4080>

# Etapa 5: Avaliação do modelo

In [ ]:
results = Dcnn.evaluate(test_dataset)
print(results)

4513/4513 [==============================] - 156s 35ms/step - loss: 0.3359 - accuracy: 0.8598
[0.33594396710395813, 0.8597731590270996]


In [ ]:
def get_prediction(sentence):
  tokens = encode_sentence(sentence)

  input_ids = get_ids(tokens)
  input_mask = get_mask(tokens)
  segment_ids = get_segments(tokens)

  inputs = tf.stack(
      [
       tf.cast(input_ids, dtype=tf.int32),
       tf.cast(input_mask, dtype=tf.int32),
       tf.cast(segment_ids, dtype=tf.int32),
      ], axis = 0)
  inputs = tf.expand_dims(inputs, 0)

  output = Dcnn(inputs, training=False)

  sentiment = math.floor(output*2)

  if sentiment == 0:
    print("Output of the model: {}\nPredicted sentiment: negative".format(output))
  elif sentiment == 1:
    print("Output of the model: {}\nPredicted sentiment: positive".format(output))


In [ ]:
get_prediction("This actor is very bad.")

Output of the model: [[0.02377077]]
Predicted sentiment: negative


In [ ]:
get_prediction("I like dogs.")

Output of the model: [[0.8251346]]
Predicted sentiment: positive
